In [2]:
import urllib3
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup

urllib3.disable_warnings() # Fix the warnings!

In [3]:
base = 'https://scholarshipdb.net/PhD-scholarships'

In [5]:
#This gets the website that we want to take in
html_text = requests.get(base, verify=False)
# Create a parser instance able to parse invalid markup.
soup = BeautifulSoup(html_text.text, 'html.parser')
#makes empty lists that will have the countries, fields and programmes
countries_all = []
fields = []
institutes = []
programmes = ([
    'PhD', 'Postdoctoral', 'Fellowship', 'Postgraduate', 
    'Master', 'Undergraduate', 'Coursework'
])
#Find all the lines that starts with 'a'
for link in soup.find_all('a'):
   #This gives you the all the URLs dound within a page's <a> tags
    url = link.get('href')
    #I am not sure 
    if url.startswith('/scholarships-in'):
        # So go through the URL from 17 to -6?
        country = url[17:-6]
        print(url)

/scholarships-in-United-States?q=PhD
/scholarships-in-United-Kingdom?q=PhD
/scholarships-in-Netherlands?q=PhD
/scholarships-in-Australia?q=PhD
/scholarships-in-Canada?q=PhD
/scholarships-in-Germany?q=PhD
/scholarships-in-Singapore?q=PhD
/scholarships-in-Sweden?q=PhD
/scholarships-in-Belgium?q=PhD
/scholarships-in-Portugal?q=PhD
/scholarships-in-Norway?q=PhD
/scholarships-in-Switzerland?q=PhD
/scholarships-in-France?q=PhD
/scholarships-in-Denmark?q=PhD
/scholarships-in-Spain?q=PhD
/scholarships-in-New-Zealand?q=PhD
/scholarships-in-Ireland?q=PhD
/scholarships-in-Poland?q=PhD
/scholarships-in-Europe?q=PhD
/scholarships-in-Austria?q=PhD
/scholarships-in-China?q=PhD
/scholarships-in-Luxembourg?q=PhD
/scholarships-in-Italy?q=PhD
/scholarships-in-United-Arab-Emirates?q=PhD
/scholarships-in-Finland?q=PhD
/scholarships-in-Czech?q=PhD
/scholarships-in-Hong-Kong?q=PhD
/scholarships-in-Croatia?q=PhD
/scholarships-in-Egypt?q=PhD
/scholarships-in-Laos?q=PhD
/scholarships-in-Suriname?q=PhD
/scholars

In [ ]:
#This gets the website that we want to take in
html_text = requests.get(base, verify=False)
# Create a parser instance able to parse invalid markup.
soup = BeautifulSoup(html_text.text, 'html.parser')
#makes empty lists that will have the countries, fields and programmes
countries_all = []
fields = []
institutes = []
programmes = ([
    'PhD', 'Postdoctoral', 'Fellowship', 'Postgraduate', 
    'Master', 'Undergraduate', 'Coursework'
])
#Find all the lines that starts with 'a'
for link in soup.find_all('a'):
    #This gives you the all the URLs dound within a page's <a> tags
    url = link.get('href')
    #I am not sure 
    if url.startswith('/scholarships-in'):
        # So go through the URL from 17 to -6?
        country = url[17:-6]
        #If there is comma in the country then split them into each of the countries
        if ',' in country:
            #Split the countries to be different for each of the , - and then extend it to the list 
            countries_all.extend(country.split(',-'))
        else:
            #Not sure what is the difference but it also appends to the country list
            countries_all.append(country)
    elif url.startswith('/scholarships?q='):
        if not url.startswith('/scholarships?q=PhD&'):
            fields.append(url[16:])

# Clean up country list
countries = []
for country in countries_all:
    if ('=' in country) or (len(country) < 2):
        break
    countries.append(country)

In [ ]:
combos = list(itertools.product(countries, programmes, fields))
print(combos)

In [ ]:
data = []
url = 'https://scholarshipdb.net/scholarships-in-{}/Program-{}?q={}'
for i, (country, programme, field) in enumerate(combos):
    html_text = requests.get(url.format(country, programme, field), verify=False)
    soup = BeautifulSoup(html_text.text, 'html.parser')
    try:
        count = int(soup.title.text.split()[0].replace(',', ''))
    except:
        count = 0
    data.append([country, programme, field, count])
    if i % 100 == 0:
        print(i)
records = pd.DataFrame(data, columns=['country', 'programme', 'field', 'count'])
records.to_csv('all-scholarships.csv', index=False)